### This notebook shapes collision data to street segment level and compute all model features needed to the same level.

1. Set-up

In [27]:
import pandas as pd
import numpy as np
from pandas import read_sql
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

2. load data 

In [25]:
# main table with case-level collision record
df = pd.read_csv('transbase_collision.csv')

# mapping table with traffic volume for each street segment
df_2 = pd.read_csv('st_sgmt_trnsprtn.csv')

In [4]:
# preview main table

df.head()

,unique_id,gc_switrs_col_pkey,cnn_intrsctn_fkey,cnn_sgmt_fkey,tb_latitude,tb_longitude,geocode_source,geocode_location,case_id_pkey,accident_year,proc_date,juris,collision_date,collision_time,time_cat,officer_id,reporting_district,day_of_week,chp_shift,population,cnty_city_loc,special_cond,beat_type,chp_beat_type,city_division_lapd,chp_beat_class,beat_number,primary_rd,secondary_rd,distance,direction,intersection,weather_1,weather_2,state_hwy_ind,caltrans_county,caltrans_district,state_route,route_suffix,postmile_prefix,postmile,location_type,ramp_intersection,side_of_hwy,tow_away,collision_severity,number_killed,number_injured,party_count,primary_coll_factor,pcf_code_of_viol,pcf_viol_category,pcf_violation,pcf_viol_subsection,hit_and_run,type_of_collision,mviw,ped_action,road_surface,road_cond_1,road_cond_2,lighting,control_device,chp_road_type,pedestrian_accident,bicycle_accident,motorcycle_accident,truck_accident,not_private_property,alcohol_involved,stwd_vehtype_at_fault,chp_vehtype_at_fault,count_severe_inj,count_visible_inj,count_complaint_pain,count_ped_killed,count_ped_injured,count_bicyclist_killed,count_bicyclist_injured,count_mc_killed,count_mc_injured,primary_ramp,secondary_ramp,latitude_switrs,longitude_switrs,mv_involved,mv_only_one,col_comb,dph_col_type,dph_col_grp,street_view
0,39,39,24142000.0,722000.0,37.765199,-122.417230,SFPD-CROSSROADS,CITY STREET,1995083,2005,20050727,3801,20050420,851,A,2152,NaN,Wednesday,Not CHP,Incorporated (over 250000),3801,Not Above,Not CHP,Not CHP,NaN,0,3D15A,16TH ST,S VAN NESS AV,74,East,Not Intersection,Cloudy,Not Stated,Not State Highway,NaN,NaN,NaN,NaN,NaN,NaN,Not State Highway,NaN,Not Stated/Not State Highway,No,Injury (Complaint of Pain),0,1,2,(Vehicle) Code Violation,Not Stated,Wrong Side of Road,21650.0,NaN,Not Hit and Run,Sideswipe,Bicycle,No Pedestrian Involved,Dry,No Unusual Condition,Not Stated,Daylight,None,0,No,Yes,No,No,Yes,No,Bicycle,Bicycle,0,0,1,0,0,0,1,0,0,-,-,NaN,NaN,NaN,NaN,1 4,H,CC,"http://maps.google.com/maps?q=&layer=c&cbll=37.7651986433249,-122.417229733255"
1,303,303,22435000.0,12774000.0,37.745936,-122.450860,SFPD-CROSSROADS,CITY STREET,2415127,2005,20060331,3801,20051112,1142,B,805,F,Saturday,Not CHP,Incorporated (over 250000),3801,Not Above,Not CHP,Not CHP,NaN,0,0F1,TWIN PEAKS,PORTOLA,40,North,Not Intersection,Clear,Not Stated,Not State Highway,NaN,NaN,NaN,NaN,NaN,NaN,Not State Highway,NaN,Not Stated/Not State Highway,No,Injury (Complaint of Pain),0,1,2,(Vehicle) Code Violation,Not Stated,Unsafe Speed,22350.0,NaN,Not Hit and Run,Broadside,Bicycle,No Pedestrian Involved,Dry,No Unusual Condition,Not Stated,Daylight,None,0,No,Yes,No,No,Yes,No,Bicycle,NaN,0,0,1,0,0,0,1,0,0,-,-,NaN,NaN,NaN,NaN,1 4,H,CC,"http://maps.google.com/maps?q=&layer=c&cbll=37.7459356551318,-122.450860384965"
2,344,344,27193000.0,NaN,37.766166,-122.460016,SFPD-CROSSROADS,CITY STREET,2624857,2006,20060708,3801,20060327,1729,C,725,F,Monday,Not CHP,Incorporated (over 250000),3801,Not Above,Not CHP,Not CHP,NaN,0,001,LINCOLN WY,KEZAR DR,0,"Not Stated, in Intersection",Intersection,Cloudy,Raining,Not State Highway,NaN,NaN,NaN,NaN,NaN,NaN,Not State Highway,NaN,Not Stated/Not State Highway,No,Injury (Complaint of Pain),0,1,2,(Vehicle) Code Violation,Not Stated,Automobile Right of Way,21804.0,A,Not Hit and Run,Broadside,Bicycle,No Pedestrian Involved,Wet,No Unusual Condition,Not Stated,Daylight,None,0,No,Yes,No,No,Yes,No,Bicycle,NaN,0,0,1,0,0,0,1,0,0,-,-,NaN,NaN,NaN,NaN,1 4,H,CC,"http://maps.google.com/maps?q=&layer=c&cbll=37.7661657875244,-122.460016419907"
3,668,669,25138000.0,5063000.0,37.783379,-122.416980,SFPD-CROSSROADS,CITY STREET,3049670,2007,20070609,3801,20070220,807,A,1970,NaN,Tuesday,Not CHP,Incorporated (over 250000),3801,Not Above,Not CHP,Not CHP,NaN,0,3J12A,EDDY ST,LARKIN,160,East,Not Intersection,Clear,Not Stated,Not State Highway,NaN,NaN,NaN,NaN,NaN,NaN,Not State Highway,NaN,Not Stated/Not State Highway,No,Injury (Complaint of Pain),0,1,2,(Vehicle) Code Violation,Not Stated,Uns

In [5]:
# preview main table

df.describe()

,unique_id,gc_switrs_col_pkey,cnn_intrsctn_fkey,cnn_sgmt_fkey,tb_latitude,tb_longitude,case_id_pkey,accident_year,proc_date,juris,collision_date,collision_time,cnty_city_loc,city_division_lapd,chp_beat_class,distance,caltrans_district,state_route,postmile,number_killed,number_injured,party_count,pcf_violation,chp_road_type,count_severe_inj,count_visible_inj,count_complaint_pain,count_ped_killed,count_ped_injured,count_bicyclist_killed,count_bicyclist_injured,count_mc_killed,count_mc_injured,latitude_switrs,longitude_switrs,mv_only_one
count,3852.000000,3852.000000,3.850000e+03,2.034000e+03,3851.000000,3851.000000,3.852000e+03,3852.000000,3.852000e+03,3852.000000,3.852000e+03,3852.000000,3852.0,0.0,3852.000000,3852.000000,73.0,73.000000,73.000000,3852.000000,3852.000000,3852.000000,3737.000000,3852.000000,3852.000000,3852.000000,3852.000000,3852.000000,3852.000000,3852.000000,3852.000000,3852.0,3852.000000,4.000000,4.000000,0.0
mean,1926.500000,1927.861111,2.595406e+07,7.088687e+06,37.769989,-122.427375,4.346931e+06,2009.019990,2.009877e+07,3806.754413,2.009088e+07,1400.841381,3801.0,NaN,0.002077,65.887072,4.0,83.972603,4.141685,0.004413,1.031412,1.956906,20441.121755,0.000260,0.066719,0.504413,0.460280,0.000519,0.016096,0.003894,1.003894,0.0,0.003115,37.704380,122.398155,NaN
std,1112.120947,1113.029458,3.927082e+06,4.084867e+06,0.019644,0.025463,1.212181e+06,2.219192,2.565125e+04,178.502936,2.218878e+04,536.821242,0.0,NaN,0.064424,477.518200,0.0,73.376687,1.771302,0.066294,0.212073,0.382982,5609.479860,0.016112,0.252669,0.515389,0.522891,0.022783,0.135784,0.062289,0.105597,0.0,0.055735,0.179047,0.094686,NaN
min,1.000000,1.000000,2.013100e+07,1.010000e+05,37.708098,-122.512871,1.984660e+06,2005.000000,2.005070e+07,3801.000000,2.005011e+07,1.000000,3801.0,NaN,0.000000,0.000000,4.0,1.000000,0.840000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,37.437000,122.262800,NaN
25%,963.750000,964.750000,2.413900e+07,3.564750e+06,37.762784,-122.438238,3.416550e+06,2007.000000,2.008031e+07,3801.000000,2.007091e+07,1000.000000,3801.0,NaN,0.000000,0.000000,4.0,1.000000,2.110000,0.000000,1.000000,2.000000,21650.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,37.686540,122.370065,NaN
50%,1926.500000,1927.500000,2.518100e+07,7.861600e+06,37.772734,-122.421377,4.469203e+06,2009.000000,2.010073e+07,3801.000000,2.009101e+07,1455.000000,3801.0,NaN,0.000000,5.000000,4.0,101.000000,4.860000,0.000000,1.000000,2.000000,21804.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,37.787060,122.427160,NaN
75%,2889.250000,2891.250000,2.664100e+07,1.017000e+07,37.782370,-122.410292,5.257571e+06,2011.000000,2.012092e+07,3801.000000,2.011062e+07,1814.000000,3801.0,NaN,0.000000,71.000000,4.0,101.000000,5.360000,0.000000,1.000000,2.000000,22350.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,37.804900,122.455250,NaN
max,3852.000000,3916.000000,5.433000e+07,1.986600e+07,37.827268,-122.369061,9.020838e+06,2012.000000,2.014051e+07,9350.000000,2.012123e+07,2359.000000,3801.0,NaN,2.000000,26400.000000,4.0,280.000000,6.976000,1.000000,5.000000,7.000000,24250.000000,1.000000,2.000000,2.000000,5.000000,1.000000,2.000000,1.000000,2.000000,0.0,1.000000,37.806400,122.475500,NaN


In [31]:
# select the cols we need

df_select = df[['case_id_pkey','cnn_sgmt_fkey', 'collision_severity', 'pcf_viol_category', 'road_cond_1']]

In [32]:
# preview selected cols

df_select.head()

,case_id_pkey,cnn_sgmt_fkey,collision_severity,pcf_viol_category,road_cond_1
0,1995083,722000.0,Injury (Complaint of Pain),Wrong Side of Road,No Unusual Condition
1,2415127,12774000.0,Injury (Complaint of Pain),Unsafe Speed,No Unusual Condition
2,2624857,NaN,Injury (Complaint of Pain),Automobile Right of Way,No Unusual Condition
3,3049670,5063000.0,Injury (Complaint of Pain),Unsafe Lane Change,No Unusual Condition
4,3216691,360000.0,Injury (Other Visible),Unsafe Speed,No Unusual Condition


3. Transform columns before aggregation
    - step 1: frequency count of all the categorical variables needed
    - step 2: convert categorical vars into numeric ones so that they can be agged down the line to segment level

In [ ]:
# 3.1 collision_severity

In [21]:
df_select['collision_severity'].value_counts()

Injury (Other Visible)        1903
Injury (Complaint of Pain)    1679
Injury (Severe)               253 
Fatal                         17  
Name: collision_severity, dtype: int64

In [ ]:
# mapping rule

# Injury (Other Visible)        4
# Injury (Complaint of Pain)    3
# Injury (Severe)               2 
# Fatal                         1

In [34]:
# convert!

df_select['cs_injury_4'] = np.where(df_select['collision_severity'] == 'Injury (Other Visible)', 1, 0)
df_select['cs_injury_3'] = np.where(df_select['collision_severity'] == 'Injury (Complaint of Pain)', 1, 0)
df_select['cs_injury_2'] = np.where(df_select['collision_severity'] == 'Injury (Injury (Severe))', 1, 0)
df_select['cs_injury_1'] = np.where(df_select['collision_severity'] == 'Injury (Fatal)', 1, 0)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [ ]:
# 3.2 pcf_viol_category

In [23]:
df_select['pcf_viol_category'].value_counts()

Improper Turning                                               615
Automobile Right of Way                                        589
Unsafe Speed                                                   510
Other Hazardous Violation                                      509
Traffic Signals and Signs                                      503
Wrong Side of Road                                             222
Improper Passing                                               163
Unknown                                                        134
Unsafe Lane Change                                             128
Not Stated                                                     128
Other Than Driver (or Pedestrian)                              98 
Unsafe Starting or Backing                                     65 
Following Too Closely                                          41 
Pedestrian Violation                                           39 
Lights                                                        

In [ ]:
# mapping rule: if associated with road attributes then 1 else 0  

# Improper Turning                                               0
# Automobile Right of Way                                        0
# Unsafe Speed                                                   0
# Other Hazardous Violation                                      0
# Traffic Signals and Signs                                      1
# Wrong Side of Road                                             0
# Improper Passing                                               0
# Unknown                                                        0
# Unsafe Lane Change                                             0
# Not Stated                                                     0
# Other Than Driver (or Pedestrian)                              0 
# Unsafe Starting or Backing                                     0 
# Following Too Closely                                          0 
# Pedestrian Violation                                           0 
# Lights                                                         0 
# Other Improper Driving                                         0 
# Pedestrian Right of Way                                        0 
# Driving or Bicycling Under the Influence of Alcohol or Drug    0 
# Impeding Traffic                                               0 
# Brakes                                                         0  
# Other Equipment                                                0  

In [33]:
# convert!

df_select['pcf_viol_category_is_st'] = np.where(df_select['pcf_viol_category'] == 'Traffic Signals and Signs', 1, 0)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# 3.3 road_cond_1

In [24]:
df_select['road_cond_1'].value_counts()

No Unusual Condition           3633
Other                          84  
Holes, Deep Ruts               50  
Not Stated                     41  
Construction or Repair Zone    22  
Loose Material on Roadway      11  
Obstruction on Roadway         8   
Reduced Roadway Width          3   
Name: road_cond_1, dtype: int64

In [ ]:
# mapping rule: only assign a code number for values associated with road characteristics

# No Unusual Condition           0
# Other                          0  
# Holes, Deep Ruts               1  
# Not Stated                     0  
# Construction or Repair Zone    2  
# Loose Material on Roadway      3  
# Obstruction on Roadway         4   
# Reduced Roadway Width          5   

In [35]:
# convert!

df_select['road_cond_new_1'] = np.where(df_select['road_cond_1'] == 'Holes, Deep Ruts', 1, 0)
df_select['road_cond_new_2'] = np.where(df_select['road_cond_1'] == 'Construction or Repair Zone', 1, 0)
df_select['road_cond_new_3'] = np.where(df_select['road_cond_1'] == 'Loose Material on Roadway', 1, 0)
df_select['road_cond_new_4'] = np.where(df_select['road_cond_1'] == 'Obstruction on Roadway', 1, 0)
df_select['road_cond_new_5'] = np.where(df_select['road_cond_1'] == 'Reduced Roadway Width', 1, 0)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

4. Merge main table and the secondary table

In [42]:
# preview main table (filtered version)

df_select.head()

,case_id_pkey,cnn_sgmt_fkey,collision_severity,pcf_viol_category,road_cond_1,pcf_viol_category_is_st,cs_injury_4,cs_injury_3,cs_injury_2,cs_injury_1,road_cond_new_1,road_cond_new_2,road_cond_new_3,road_cond_new_4,road_cond_new_5
0,1995083,722000.0,Injury (Complaint of Pain),Wrong Side of Road,No Unusual Condition,0,0,1,0,0,0,0,0,0,0
1,2415127,12774000.0,Injury (Complaint of Pain),Unsafe Speed,No Unusual Condition,0,0,1,0,0,0,0,0,0,0
2,2624857,NaN,Injury (Complaint of Pain),Automobile Right of Way,No Unusual Condition,0,0,1,0,0,0,0,0,0,0
3,3049670,5063000.0,Injury (Complaint of Pain),Unsafe Lane Change,No Unusual Condition,0,0,1,0,0,0,0,0,0,0
4,3216691,360000.0,Injury (Other Visible),Unsafe Speed,No Unusual Condition,0,1,0,0,0,0,0,0,0,0


In [50]:
# filter secondary table with cols needed

df_2_select = df_2[['cnn_sgmt_pkey', 'daily_ride_qrt', 'daily_ride_eght', 'speed_avg_mta']]

In [51]:
# preview filtered version
# speed_avg_mta has a lot of null values

df_2_select.describe()

,cnn_sgmt_pkey,daily_ride_qrt,daily_ride_eght,speed_avg_mta
count,1.549000e+04,15062.000000,15062.000000,600.000000
mean,7.826860e+06,44178.461957,13401.134245,27.715000
std,4.846887e+06,44459.348097,16551.960535,5.282071
min,1.000000e+05,40.000000,0.000000,13.000000
25%,3.846250e+06,14135.000000,2329.000000,25.000000
50%,7.655500e+06,30380.500000,7690.000000,27.000000
75%,1.121975e+07,59413.250000,18133.000000,31.250000
max,2.000500e+07,330128.000000,154916.000000,47.000000


In [53]:
# merge

df_3 = df_select.merge(df_2_select, left_on = 'cnn_sgmt_fkey', right_on = 'cnn_sgmt_pkey', how = 'inner')

In [54]:
df_3.describe()

,case_id_pkey,cnn_sgmt_fkey,pcf_viol_category_is_st,cs_injury_4,cs_injury_3,cs_injury_2,cs_injury_1,road_cond_new_1,road_cond_new_2,road_cond_new_3,road_cond_new_4,road_cond_new_5,cnn_sgmt_pkey,daily_ride_qrt,daily_ride_eght,speed_avg_mta
count,2.034000e+03,2.034000e+03,2034.000000,2034.000000,2034.000000,2034.0,2034.0,2034.000000,2034.000000,2034.000000,2034.000000,2034.000000,2.034000e+03,2029.000000,2029.000000,144.000000
mean,4.322134e+06,7.088687e+06,0.045723,0.505900,0.421337,0.0,0.0,0.014749,0.006391,0.004425,0.003441,0.001475,7.088687e+06,85768.108921,28642.857565,27.506944
std,1.209503e+06,4.084867e+06,0.208934,0.500088,0.493895,0.0,0.0,0.120577,0.079710,0.066388,0.058578,0.038386,4.084867e+06,64243.512737,27088.134060,4.967019
min,1.984660e+06,1.010000e+05,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.010000e+05,140.000000,0.000000,18.000000
25%,3.416608e+06,3.564750e+06,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.564750e+06,37500.000000,9307.000000,25.000000
50%,4.445733e+06,7.861600e+06,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,7.861600e+06,68459.000000,20679.000000,27.000000
75%,5.238910e+06,1.017000e+07,0.000000,1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.017000e+07,112695.000000,39007.000000,31.000000
max,9.020822e+06,1.986600e+07,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.986600e+07,325565.000000,141964.000000,38.000000


In [55]:
df_3.head()

,case_id_pkey,cnn_sgmt_fkey,collision_severity,pcf_viol_category,road_cond_1,pcf_viol_category_is_st,cs_injury_4,cs_injury_3,cs_injury_2,cs_injury_1,road_cond_new_1,road_cond_new_2,road_cond_new_3,road_cond_new_4,road_cond_new_5,cnn_sgmt_pkey,daily_ride_qrt,daily_ride_eght,speed_avg_mta
0,1995083,722000.0,Injury (Complaint of Pain),Wrong Side of Road,No Unusual Condition,0,0,1,0,0,0,0,0,0,0,722000,103647.0,36476.0,NaN
1,6059168,722000.0,Injury (Complaint of Pain),Wrong Side of Road,No Unusual Condition,0,0,1,0,0,0,0,0,0,0,722000,103647.0,36476.0,NaN
2,2415127,12774000.0,Injury (Complaint of Pain),Unsafe Speed,No Unusual Condition,0,0,1,0,0,0,0,0,0,0,12774000,26810.0,15393.0,NaN
3,3049670,5063000.0,Injury (Complaint of Pain),Unsafe Lane Change,No Unusual Condition,0,0,1,0,0,0,0,0,0,0,5063000,172089.0,43865.0,NaN
4,2774877,5063000.0,Injury (Other Visible),Other Hazardous Violation,No Unusual Condition,0,1,0,0,0,0,0,0,0,0,5063000,172089.0,43865.0,NaN


5. group by segment key and agg all features to segment level

In [58]:
df_sgmt = df_3.groupby('cnn_sgmt_fkey').agg({'case_id_pkey':'size',
                                   'pcf_viol_category_is_st':'sum',
                                   'cs_injury_4':'sum',
                                   'cs_injury_3':'sum',
                                   'cs_injury_2':'sum',
                                   'cs_injury_1':'sum',
                                   'road_cond_new_1':'sum',
                                   'road_cond_new_2':'sum',
                                   'road_cond_new_3':'sum',
                                   'road_cond_new_4':'sum',
                                   'road_cond_new_5':'sum',
                                   'daily_ride_qrt':'sum',
                                   'daily_ride_eght':'sum',
                                   'speed_avg_mta':'mean'                                        
                                       }).reset_index()

In [62]:
df_sgmt.head(10)

,cnn_sgmt_fkey,case_id_pkey,pcf_viol_category_is_st,cs_injury_4,cs_injury_3,cs_injury_2,cs_injury_1,road_cond_new_1,road_cond_new_2,road_cond_new_3,road_cond_new_4,road_cond_new_5,daily_ride_qrt,daily_ride_eght,speed_avg_mta
0,101000.0,1,0,1,0,0,0,0,0,0,0,0,124386.0,27553.0,NaN
1,104002.0,1,0,1,0,0,0,0,0,0,0,0,82433.0,24254.0,NaN
2,129000.0,3,1,3,0,0,0,0,0,0,0,0,539595.0,219624.0,NaN
3,130000.0,1,0,0,1,0,0,0,0,0,0,0,189071.0,73208.0,NaN
4,131000.0,1,0,1,0,0,0,0,0,0,0,0,179302.0,67625.0,NaN
5,133000.0,1,0,1,0,0,0,0,0,0,0,0,163574.0,14796.0,NaN
6,134000.0,1,0,1,0,0,0,0,0,0,0,0,157739.0,9195.0,NaN
7,136000.0,2,0,1,1,0,0,0,0,0,0,0,111100.0,23370.0,NaN
8,139000.0,1,0,0,1,0,0,0,0,0,0,0,62002.0,13781.0,NaN
9,140000.0,1,0,0,0,0,0,0,0,0,0,0,54567.0,18544.0,NaN


In [60]:
df_sgmt.describe()

,cnn_sgmt_fkey,case_id_pkey,pcf_viol_category_is_st,cs_injury_4,cs_injury_3,cs_injury_2,cs_injury_1,road_cond_new_1,road_cond_new_2,road_cond_new_3,road_cond_new_4,road_cond_new_5,daily_ride_qrt,daily_ride_eght,speed_avg_mta
count,1.349000e+03,1349.000000,1349.000000,1349.000000,1349.000000,1349.0,1349.0,1349.000000,1349.000000,1349.000000,1349.000000,1349.000000,1.349000e+03,1.349000e+03,81.000000
mean,6.659512e+06,1.507784,0.068940,0.762787,0.635285,0.0,0.0,0.022239,0.009637,0.006672,0.005189,0.002224,1.290018e+05,4.308107e+04,28.456790
std,4.151284e+06,1.249085,0.264895,0.914802,0.777350,0.0,0.0,0.157250,0.097729,0.081437,0.071874,0.047123,1.972845e+05,8.093916e+04,4.257492
min,1.010000e+05,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,18.000000
25%,3.072000e+06,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.750000e+04,8.958000e+03,26.000000
50%,6.813000e+06,1.000000,0.000000,1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,7.289000e+04,2.257400e+04,28.000000
75%,9.813101e+06,2.000000,0.000000,1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.524180e+05,4.537500e+04,31.000000
max,1.986600e+07,19.000000,2.000000,9.000000,8.000000,0.0,0.0,2.000000,1.000000,1.000000,1.000000,1.000000,2.365650e+06,1.213670e+06,38.000000


6.  compute y

In [65]:
# Number of collision/total_trips (measured by daily_ride_qrt)
df_sgmt['y_1'] = df_sgmt['case_id_pkey'] * 1. / df_sgmt['daily_ride_qrt']

# Number of collision/total_trips (measured by daily_ride_eght)
df_sgmt['y_2'] = df_sgmt['case_id_pkey'] * 1. / df_sgmt['daily_ride_eght']

In [66]:
df_sgmt.head(20)

,cnn_sgmt_fkey,case_id_pkey,pcf_viol_category_is_st,cs_injury_4,cs_injury_3,cs_injury_2,cs_injury_1,road_cond_new_1,road_cond_new_2,road_cond_new_3,road_cond_new_4,road_cond_new_5,daily_ride_qrt,daily_ride_eght,speed_avg_mta,y_1,y_2
0,101000.0,1,0,1,0,0,0,0,0,0,0,0,124386.0,27553.0,NaN,0.000008,0.000036
1,104002.0,1,0,1,0,0,0,0,0,0,0,0,82433.0,24254.0,NaN,0.000012,0.000041
2,129000.0,3,1,3,0,0,0,0,0,0,0,0,539595.0,219624.0,NaN,0.000006,0.000014
3,130000.0,1,0,0,1,0,0,0,0,0,0,0,189071.0,73208.0,NaN,0.000005,0.000014
4,131000.0,1,0,1,0,0,0,0,0,0,0,0,179302.0,67625.0,NaN,0.000006,0.000015
5,133000.0,1,0,1,0,0,0,0,0,0,0,0,163574.0,14796.0,NaN,0.000006,0.000068
6,134000.0,1,0,1,0,0,0,0,0,0,0,0,157739.0,9195.0,NaN,0.000006,0.000109
7,136000.0,2,0,1,1,0,0,0,0,0,0,0,111100.0,23370.0,NaN,0.000018,0.000086
8,139000.0,1,0,0,1,0,0,0,0,0,0,0,62002.0,13781.0,NaN,0.000016,0.000073
9,140000.0,1,0,0,0,0,0,0,0,0,0,0,54567.0,18544.0,NaN,0.000018,0.000054


In [67]:
df_sgmt.to_csv('df_sgmt_agg_20191106.csv', index = False)